In [4]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math

import tensorflow as tf

import qgrid

In [5]:
df=pd.read_csv('c:/temp/final1_20240619.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Change
0,2023-01-02,55500,56100,55200,55500,10031448,0.003617
1,2023-01-03,55400,56000,54500,55400,13547030,-0.001802
2,2023-01-04,55700,58000,55600,57800,20188071,0.043321
3,2023-01-05,58200,58800,57600,58200,15682826,0.006920
4,2023-01-06,58300,59400,57900,59000,17334989,0.013746


In [6]:
df.tail()

,Date,Open,High,Low,Close,Volume,Change
240,2023-12-21,74600,75000,74300,75000,13478766,0.002674
241,2023-12-22,75800,76300,75400,75900,14515608,0.012000
242,2023-12-26,76100,76700,75700,76600,13164909,0.009223
243,2023-12-27,76700,78000,76500,78000,20651042,0.018277
244,2023-12-28,77700,78500,77500,78500,17797536,0.006410


In [7]:
# daily return(ret) 변수 생성, 평균 구해 소수점 셋째자리, 백분위 퍼센트로 작성할 것

In [8]:
# return (r): 수익률
# 양수면 주가 상승, 음수면 주가 하락
# python으로 수익률 r 계산가능
df['ret']=100*(df['Close'].pct_change())
df.head()

,Date,Open,High,Low,Close,Volume,Change,ret
0,2023-01-02,55500,56100,55200,55500,10031448,0.003617,NaN
1,2023-01-03,55400,56000,54500,55400,13547030,-0.001802,-0.180180
2,2023-01-04,55700,58000,55600,57800,20188071,0.043321,4.332130
3,2023-01-05,58200,58800,57600,58200,15682826,0.006920,0.692042
4,2023-01-06,58300,59400,57900,59000,17334989,0.013746,1.374570


In [13]:

# 수익률 나타내는 ret 칼럼 생김
# 2021-01-04의 ret는 결측치일 수 밖에 없음, 앞에 데이터가 없으니까
df1=df['ret']
r=df1.dropna()
# r: 수익률만 있는 칼럼 = 247일 동안 주가가 오르고 내린게 기록되어 있는 것
r. describe() # r에 대한 통계량 알 수 있음

count    244.000000
mean       0.150921
std        1.327962
min       -3.633491
25%       -0.694461
50%        0.137646
75%        0.805842
max        6.128550
Name: ret, dtype: float64

In [14]:


# 2021년
# 평균 수익률 -0.01% -> 거의 0
# 근데 알고 싶은 것: 247일 동안 몇일 동안 야으이 수익률 보이는 가
# + 값 몇 개인지 세기 : (r>0).sum()
positive_count=(r>0).sum()
positive_count


124

In [15]:
# 즉 1년동안 데이터로 추정한 '주가가 오를 확률' = p
124/244

0.5081967213114754

In [16]:
prob3=math.comb(10,3)*(0.5081967213114754**3)*(1-0.5081967213114754)**7
prob2=math.comb(10,2)*(0.5081967213114754**2)*(1-0.5081967213114754)**8
prob1=math.comb(10,1)*(0.5081967213114754**1)*(1-0.5081967213114754)**9
prob0=math.comb(10,0)*(0.5081967213114754**0)*(1-0.5081967213114754)**10
print(prob0+prob1+prob2+prob3)

0.15875781906349132


In [17]:
# 1 단계: 상자의 분산인 varX 계산
varX=0.5081967213114754-(0.5081967213114754**2)
varX

0.24993281375974202

In [18]:
# 2 단계: 상자의 표준편차 sdX 계산
sdX=np.sqrt(varX)
sdX

0.4999328092451445

In [19]:
# 3 단계: 합의 표준오차 계산
# 합의 표준오차 = 상자의 표준편차*sqrt(시행횟수)
se=sdX*np.sqrt(100)
se

4.999328092451445